In [1]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import functions, Row
from decimal import Decimal
from datetime import datetime

In [2]:
appid = "f7a784e8-9194-4816-872f-d1439502da4f"
appkey = "9804J-7ko39Wz9PX2_ak~Pw-4wis6tnEDK"
tenantid = "72f988bf-86f1-41af-91ab-2d7cd011db47"

container = "commondataservice-miqenv-org318511a8"
storageAccountName = "amldemosa.dfs.core.windows.net"

In [3]:
dbutils.fs.mount(
  source = "wasbs://commondataservice-miqenv-org318511a8@amldemosa.blob.core.windows.net",
  mount_point = "/mnt/airbnbreferencedata",
  extra_configs = {"fs.azure.account.key.amldemosa.blob.core.windows.net":dbutils.secrets.get(scope = "secret-scope", key = "amldemosakey")})

--------------------------------------------------------------------------- 
 ExecutionError Traceback (most recent call last)
 <command-2414188787837658> in <module> 
 2 source = "wasbs://commondataservice-miqenv-org318511a8@amldemosa.blob.core.windows.net" , 
 3 mount_point = "/mnt/airbnbreferencedata" , 
 ----> 4 extra_configs = {"fs.azure.account.key.amldemosa.blob.core.windows.net":dbutils.secrets.get(scope = "secret-scope", key = "amldemosakey")})
 
 /local_disk0/tmp/1592470213550-0/dbutils.py in f_with_exception_handling (*args, **kwargs) 
 312 exc . __context__ = None 
 313 exc . __cause__ = None 
 --> 314 raise exc
 315 return f_with_exception_handling
 316 

 ExecutionError : An error occurred while calling o298.mount.
: java.rmi.RemoteException: java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/airbnbreferencedata; nested exception is: 
	java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/airbnbreferencedata
	at com.databricks.backend.daemon.data.client.DbfsClient.send0(DbfsClient.scala:123)
	at com.databricks.backend.daemon.data.client.DbfsClient.sendIdempotent(DbfsClient.scala:63)
	at com.databricks.backend.daemon.dbutils.DBUtilsCore.mount(DBUtilsCore.scala:465)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)
Caused by: java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/airbnbreferencedata
	at scala.Predef$.require(Predef.scala:281)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.$anonfun$insertMount$1(MetadataManager.scala:224)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.withRetries(MetadataManager.scala:326)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.insertMount(MetadataManager.scala:220)
	at com.databricks.backend.daemon.data.server.handler.MountHandler.receive(MountHandler.scala:79)
	at com.databricks.backend.daemon.data.server.session.SessionContext.$anonfun$queryHandlers$1(SessionContext.scala:103)
	at com.databricks.backend.daemon.data.server.session.SessionContext.$anonfun$queryHandlers$1$adapted(SessionContext.scala:102)
	at scala.collection.immutable.List.foreach(List.scala:392)
	at com.databricks.backend.daemon.data.server.session.SessionContext.queryHandlers(SessionContext.scala:102)
	at com.databricks.backend.daemon.data.server.DbfsServerBackend$$anonfun$receive$3.applyOrElse(DbfsServerBackend.scala:337)
	at com.databricks.backend.daemon.data.server.DbfsServerBackend$$anonfun$receive$3.applyOrElse(DbfsServerBackend.scala:312)
	at com.databricks.rpc.ServerBackend.$anonfun$internalReceive$2(ServerBackend.scala:51)
	at com.databricks.rpc.ServerBackend$$anonfun$commonReceive$1.applyOrElse(ServerBackend.scala:78)
	at com.databricks.rpc.ServerBackend$$anonfun$commonReceive$1.applyOrElse(ServerBackend.scala:78)
	at com.databricks.rpc.ServerBackend.$anonfun$internalReceive$1(ServerBackend.scala:47)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$4(UsageLogging.scala:430)
	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:238)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.UsageLogging.withAttributionContext(Usage

In [4]:
df = (spark.read.format("com.microsoft.cdm")
  .option("storage", storageAccountName)
  .option("container", container)
  .option("manifest", "/model.json")
  .option("entity", "cr78b_reference1")
  .option("appId", appid)
  .option("appKey", appkey)
  .option("tenantId", tenantid)
  .load())


In [5]:
keepcolumns =["cr78b_quality", "cr78b_neighborhood","cr78b_distance_from_airport"]
new = df[keepcolumns]
new1 = new.withColumnRenamed("cr78b_quality", "quality")\
       .withColumnRenamed("cr78b_neighborhood", "neighborhood")\
       .withColumnRenamed("cr78b_distance_from_airport", "distance_from_airport")

new2 = new1.withColumn("distance_from_airport",new1["distance_from_airport"].cast(DecimalType()))
display(new2)

quality,neighborhood,distance_from_airport
excellent,Centrum West,8
excellent,De Pijp / Rivierenbuurt,5
good,De Baarsjes / Oud West,6
very bad,Watergraafsmeer,4
very good,Westerpark,7
bad,Oostelijk Havengebied / Indische Buurt,9
very good,Centrum Oost,8
good,Oud Oost,8
good,Buitenveldert / Zuidas,3
terrible,Geuzenveld / Slotermeer,4


In [6]:
(new2.write.format("com.microsoft.cdm")
  .option("storage", storageAccountName)
  .option("container", "amldemocontainer")
  .option("manifest", "/withdecimal/default.manifest.cdm.json")
  .option("entity", "Referencewiyhdecimal")
  .option("format", "parquet")
  .option("appId", appid)
  .option("appKey", appkey)
  .option("tenantId", tenantid)
  .save())